# 第二节 解析 Transformer 的构造

我们将使用自然语言处理（NLP）的任务，逐步拆解Transformer的结构进行讲解。

本节的 notebook 参考自：https://github.com/nlp-with-transformers/notebooks?tab=readme-ov-file , chapter 03

本节不涉及使用模型训练的环节。

**如果在 Google Colab 中使用，请首先运行下面的命令安装相关的python包。**


In [ ]:
! pip install bertviz

## 前言：为什么要用 Transformer？

当我们设计的模型越来越大，处理的任务越来越复杂，Transformer 就会表现出无可替代的优势。Transformer 的但是直接催生了大语言模型的成功（GPT和BERT）。目前在各个领域，具有最高性能的模型基本都是基于 Transformer 架构的。在高能物理的应用中，也是如此。

下面举一个喷注标定（jet tagging）的例子：

在 ATLAS 和 CMS 实验组中，小半径的（R=0.4）喷注味道鉴别任务是几乎半数分析都会使用的。这主要涉及到利用一个喷注的所有底层组分粒子（大概有o(30)个）的信息，用 DNN 判断喷注是否来自于 b 夸克；部分物理分析也需要判断喷注是否来自于 c 夸克、轻 (u/d/s) 夸克，或者胶子。这样的 DNN 工具被称为 jet tagger。因为喷注鉴别的通用型，训练和部署 jet tagger 是一个实验组的中心任务。这个分类问题也是相较物理分析而言更复杂的问题。可以看到，随着 DNN 的设计和训练技术的发展，目前的 ATLAS 和 CMS 最检验的 jet tagger 都已经更迭到了 Transformer 架构：ATLAS 的最新的 tagger 是 [GN2](https://atlas.web.cern.ch/Atlas/GROUPS/PHYSICS/PLOTS/FTAG-2023-01/)、CMS 是 [UParT](https://cds.cern.ch/record/2904702/files/DP2024_066.pdf)。CMS 的 tagger 在b-tagging压低轻味道 jet 本底方面，相比 Run-2 早期阶段已经提升了50倍！图中可以看出，目前尚未有达到瓶颈的趋势。这说明对于复杂任务（且有大量数据集的情况下），想要模型表现更好，选择 Transformer 是当前必须的选择——当然，如何设计、如何训练，是一个重要的AI工程的问题！我们后面第4节就会讨论设计Transformer方面的一些insights）

<img src="figures/transformers-for-jet-tagging.png" alt="image" width=900/>

不过在次之前，我们在第2、3节主要还是从基础的 Transformer 开始上手搭建，并用简单的分类任务来做些实验进行学习。

## 背景回顾

Transformer 自2017年诞生，已经“横扫”各个领域，在视觉和文本任务中性能纷纷超过了之前的 CNN 和 RNN 网络架构，成为“大一统”的新型网络架构。除了机器学习领域的任务，Transformer架构也被大量用在科学领域。

它是为了处理拥有两个维度或更高维度数据而生的。这里“二维数据”指：有很多token，每个token是一个特征向量作为输入。

 - 例如对图像：token可以是16x16的像素块，token特征是对这个小图像块的潜空间的特征
 - 例如对文本：token可以是基础文本token，特征就是词嵌入特征。

**用如下例子可以进行形象理解**（该比喻将贯穿我们的教程）：“二维数据”就像是教室里的一组同学。每个学生自己具有一个特征向量（想象成脑中的一种知识理解的状态），而数据的整体是所有同学构成的。对应到上面的图像/文本的例子中，每个学生对应为一个像素块/一个单词。

容易想见，传统的 DNN 网络（习惯上称为 multi-layer preceptron, MLP）即适用于处理“一维数据”。
对于内禀有二维结构的数据，当然可以“拉直”变成一维的向量，再交给DNN处理。但是，这丢失了本身具有的结构，网络学习不够高效，性能不会很好。
Transformer则即让token这一单一维度进行MLP操作，又能高效地学习到这些token之间的联系（通过自注意力机制），从而变成处理图像、文本等等一系列复杂数据类型的最佳的网络架构。

## 基本结构

Tranformer这种架构，怎样处理各个token（每个同学脑海的知识），从而高效地学到信息呢？

视频中展示了Transformer具有以下结构。视频详细表述了1和3，2将是本节重点。
1. 对初始token进行嵌入（embedding）——即每个同学自己用相同的流程更新脑海的知识。嵌入的目的是从最原始的输入进行非线性变换，嵌入到潜空间中，使得新的矢量更具备抽象信息（例如单词意思接近的，其潜空间矢量也相近，等等）
2.  对嵌入后的token，连续进行多次Transformer的单元模块（多头自注意力, multi-head self-attention）。self-attention可理解为高效的沟通的机制，也是本节的重点。按视频介绍的，分两步走：
  - 多头自注意力——这是沟通的核心（attention部分）
  - 进行两次非线性变换，抽取其抽象特征（feed-forward network, FFN部分）
3. 对完成多次Transformer模块的token，作最后的unembbeding操作。相当于最后汇总大家的知识，形成网络的输出。

下图展示了每个Transformer模块里面的两部分组成。


<img alt="encoder-zoom" caption="Zooming into the encoder layer" src="figures/encoder-zoom.png?raw=1" id="encoder-zoom"/>

## 搭建一个 Transformer 模块

考虑教室里每名同学是一个token。他们是如何进行一次“沟通”操作，更新自己的知识呢？

 - 首先，经过一个线性变换层，将当前同学的特征向量（一个1维，具有固定长度的向量）转换为三个不同的特征向量（query, key, value）；
 - 每个同学开始和其它所有同学交流。交流方式是，用自己的 query 向量点乘其它同学的 key 向量。这样，每个同学都和其它同学以一个点积的结果连结起来；
 - 对每个同学而言，他和所有N个同学（包括自己）有了N个点积。把这些点积过 softmax (先作e指数后归一），就得到了N个weight。这就是N个同学对自己的“重要性“；
 - 用这些 weight 来加权每个同学的 value 向量，就是每个同学自己更新后的向量了。

完成这样一次操作后，每个同学的向量，变成了各个同学的向量的某种加权求和。以上就是self-attention方法的理解。

在沟通完成后，经过一次FFN的操作，进行非线性的映射，从而在这次交流中学习到信息。

### 举例

下面的两段文本中，flies分别代表"时间的流逝"和“蚊子”。
- 在self-attention阶段，各个单词（token）先相互交流，各自探讨一下对彼此重要的token是什么，然后对这些重要的token进行加权。
- 在FFN阶段，通过复杂的非线性映射，提取出一些抽象的特征。形象理解的话，可以抽取到这样的特征：这个单词是动词吗？它是形容时间的吗？它是形容某种动物吗？等等

这样的Transformer块多次进行，逐渐提取越来越抽象和高级的特征，构建一个深度的Transformer网络。



<img alt="Contextualized embeddings" caption="Diagram showing how self-attention updates raw token embeddings (upper) into contextualized embeddings (lower) to create representations that incorporate information from the whole sequence" src="figures/contextualized-embedding.png?raw=1" id="contextualized-embeddings"/>

可以利用下面对BERT（一个早期NLP Transformer）的每一个Transformer模块中，每个token之间的重要性（weight）进行可视化，观察下每一层的Transformer block里各token如何交流信息。

注：如果不是在Google Colab运行，需要把下面这段代码拷贝到一个单独jupyter block里运行一下，加载插件。Google Colab用户可忽视。

```python
%%javascript
require.config({
  paths: {
      d3: '//cdnjs.cloudflare.com/ajax/libs/d3/3.4.8/d3.min',
      jquery: '//ajax.googleapis.com/ajax/libs/jquery/2.0.0/jquery.min',
  }
});
```

In [ ]:
from transformers import AutoTokenizer
from bertviz.transformers_neuron_view import BertModel
from bertviz.neuron_view import show

model_ckpt = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = BertModel.from_pretrained(model_ckpt)
text = "time flies like an arrow"
show(model, "bert", tokenizer, text, display_mode="light", layer=0, head=8)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

 97%|█████████▋| 426056704/440473133 [03:18<00:17, 818791.59B/s]

### 解析多头自注意力里 query, key, value 的操作

<img alt="Operations in scaled dot-product attention" height="125" caption="Operations in scaled dot-product attention" src="figures/attention-ops.png?raw=1" id="attention-ops"/>

准备工作：第一步，初始embedding。这里载入的训练好的BERT模型里的embedding layer。

可以看到，embed前的token是一个30522维的向量（每个token都是单位矢量(0,...0,1,0,...,0)，对应于0--30521的整数编码），而embed后的向量维数是768。

In [5]:
from transformers import AutoTokenizer
model_ckpt = "bert-base-uncased"
text = "time flies like an arrow"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

In [6]:
inputs = tokenizer(text, return_tensors="pt", add_special_tokens=False)
inputs.input_ids

tensor([[ 2051, 10029,  2066,  2019,  8612]])

In [3]:
from torch import nn
from transformers import AutoConfig

config = AutoConfig.from_pretrained(model_ckpt)
token_emb = nn.Embedding(config.vocab_size, config.hidden_size)
token_emb

Embedding(30522, 768)

In [7]:
inputs_embeds = token_emb(inputs.input_ids)
inputs_embeds.size()

torch.Size([1, 5, 768])

下面是自注意力操作。首先由当前embed后的token向量得到query，key和value。这里先作简单考虑，认为三者直接等于embeded向量。

第$i$个同学脑海中三个向量：$q_i$, $k_i$和$v_i$。用数学表示，两个同学$i$和$j$的自注意力值即它们的$q$和$k$的点积（再除以一个归一化系数）

${\rm score}_{ij} = \frac{q_i\cdot k_j}{\sqrt{d}}$

用矩阵乘法写出上式。$Q$和$K$分别是$N\times d$的向量，$N$是token个数，$d$是每个token维数。点积即在第二个维度上作乘法，因此写作

${\rm score} = \frac{Q\cdot K^T}{\sqrt{d}}$

> $Q$和$K$可以在最前还有一个$B$维度，是批量训练的时候的批维度。因此使用`torch.bmm`作批矩阵乘法：第一个维度不变，对后两个维度进行矩阵乘法操作。
>
> `scores`的维度应该为 $B\times N\times N$。

In [8]:
import torch
from math import sqrt

query = key = value = inputs_embeds
dim_k = key.size(-1)
scores = torch.bmm(query, key.transpose(1,2)) / sqrt(dim_k)
scores.size()

torch.Size([1, 5, 5])

现在每个同学（第$i$个同学），都有和所有同学（第$j$个同学）计算所得的score。我们对这个`score`进行softmax归一化，即在$j$指标这个维度上归一化，得到`weights`

`weights` 维数性质不变。可以看到，在$j$指标求和均为1.

In [10]:
import torch.nn.functional as F

weights = F.softmax(scores, dim=-1)
print(f'{weights=}')
print(f'{weights.sum(dim=-1)=}')

weights=tensor([[[1.0000e+00, 1.9177e-12, 6.0688e-13, 2.1290e-12, 4.8709e-12],
         [5.8960e-12, 1.0000e+00, 2.8365e-12, 4.0444e-13, 7.4422e-12],
         [2.0494e-13, 3.1156e-13, 1.0000e+00, 2.4241e-12, 9.0828e-14],
         [6.4239e-13, 3.9692e-14, 2.1659e-12, 1.0000e+00, 4.5371e-13],
         [4.1308e-13, 2.0528e-13, 2.2809e-14, 1.2752e-13, 1.0000e+00]]],
       grad_fn=<SoftmaxBackward0>)
weights.sum(dim=-1)=tensor([[1., 1., 1., 1., 1.]], grad_fn=<SumBackward1>)


下面要利用这个`weights`，对所有的$v_{j}$加权求和，即对于第$i$个同学，其新的特征为

$v_{i}' = \sum_{j} w_{ij}v_{j}$

依然可以用`torch.bmm`用矩阵运算得出所有同学$i$的特征。

In [11]:
attn_outputs = torch.bmm(weights, value)
print(f'{attn_outputs.shape=}')

attn_outputs.shape=torch.Size([1, 5, 768])


合起来如下所示

In [15]:
def scaled_dot_product_attention(query, key, value):
    dim_k = query.size(-1)
    scores = torch.bmm(query, key.transpose(1, 2)) / sqrt(dim_k)
    weights = F.softmax(scores, dim=-1)
    return torch.bmm(weights, value)

### 多头注意力 (Multi-head attention) 的部分

真实情况下，query, key和value是通过当前的token，各自过一层不同的线性层实现的。单线性层的主要目的就是改变特征向量的长度，同时使得三者各不相同。

而上述的所有操作，在Transformer模块里可以实现多次，这就是”多头“的含义。可以看出，每次实现相当于一次“沟通”成果。多头自注意力即包含多次沟通成果——即学到不同的$w_{ij}$。
在多头（头数=$h$）的具体实现中，总的向量维数是不改变的，因此每个头的query, key和value的长度将是单头情形的$1/h$倍。最后每个头得到的value再串接起来，恢复到原先的维度。

下面是单个注意力头的实现，和多头注意力的实现。


<img alt="Multi-head attention" height="125" caption="Multi-head attention" src="figures/multihead-attention.png?raw=1" id="multihead-attention"/>

In [12]:
class AttentionHead(nn.Module):
    def __init__(self, embed_dim, head_dim):
        super().__init__()
        self.q = nn.Linear(embed_dim, head_dim)
        self.k = nn.Linear(embed_dim, head_dim)
        self.v = nn.Linear(embed_dim, head_dim)

    def forward(self, hidden_state):
        attn_outputs = scaled_dot_product_attention(
            self.q(hidden_state), self.k(hidden_state), self.v(hidden_state))
        return attn_outputs

In [13]:
class MultiHeadAttention(nn.Module):
    def __init__(self, config):
        super().__init__()
        embed_dim = config.hidden_size
        num_heads = config.num_attention_heads
        head_dim = embed_dim // num_heads
        self.heads = nn.ModuleList(
            [AttentionHead(embed_dim, head_dim) for _ in range(num_heads)]
        )
        self.output_linear = nn.Linear(embed_dim, embed_dim)

    def forward(self, hidden_state):
        x = torch.cat([h(hidden_state) for h in self.heads], dim=-1)
        x = self.output_linear(x)
        return x

下面就是进行一次多头自注意力操作后，更新的token特征了。

In [16]:
multihead_attn = MultiHeadAttention(config)
attn_output = multihead_attn(inputs_embeds)
attn_output.size()

torch.Size([1, 5, 768])

In [17]:
from bertviz import head_view
from transformers import AutoModel

model = AutoModel.from_pretrained(model_ckpt, output_attentions=True)

sentence_a = "time flies like an arrow"
sentence_b = "fruit flies like a banana"

viz_inputs = tokenizer(sentence_a, sentence_b, return_tensors='pt')
attention = model(**viz_inputs).attentions
sentence_b_start = (viz_inputs.token_type_ids == 0).sum(dim=1)
tokens = tokenizer.convert_ids_to_tokens(viz_inputs.input_ids[0])

head_view(attention, tokens, sentence_b_start, heads=[8])

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

BertSdpaSelfAttention is used but `torch.nn.functional.scaled_dot_product_attention` does not support non-absolute `position_embedding_type` or `output_attentions=True` or `head_mask`. Falling back to the manual attention implementation, but specifying the manual implementation will be required from Transformers version v5.0.0 onwards. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.


<IPython.core.display.Javascript object>

### 前馈神经网络 (FFN) 的部分

FFN一般仅是一个两层MLP构成的简单网络：它有两层线性变换层，中间添加非线性激活函数；同时，中间的隐藏特征维度大于token的维度。理解上看，它相当于扩展到更高维度空间中作非线性变换，再投影回原来的维度空间。这样的结构往往简单但具有较强的非线性拟合能力。

> 注：Transformer的非线性激活函数常用GELU，相比ReLU在模型最终表现上会更强。

In [19]:
class FeedForward(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.linear_1 = nn.Linear(config.hidden_size, config.intermediate_size)
        self.linear_2 = nn.Linear(config.intermediate_size, config.hidden_size)
        self.gelu = nn.GELU()
        self.dropout = nn.Dropout(config.hidden_dropout_prob)

    def forward(self, x):
        x = self.linear_1(x)
        x = self.gelu(x)
        x = self.linear_2(x)
        x = self.dropout(x)
        return x

In [20]:
feed_forward = FeedForward(config)
ff_outputs = feed_forward(attn_outputs)
ff_outputs.size()

torch.Size([1, 5, 768])

### 层归一化 (LayerNorm)

在注意力部分和FFN部分前后和中间常需要添加归一化操作。这些操作不带可学习的参数，仅为了把token的特征向量（每个同学自己的向量）各自做一个平移、缩放，使得该向量的d个值的平均值为0，方差为1。这种标准化方案是是自然语言处理的通用方案（解决了一些训练不稳定的问题，同时可以更适配于token数量可变的情况），也是Transformer的标准方案。


### 残差连接

早期的Residue neural network (ResNet) ，帮助网络可以层数训练地更深而不会崩溃，开启了近10年的DNN时代。这种残差连接的方式也是现代Transformer架构的基石。

下图展现了LayerNorm和加入残差连接后的路线图。

<img alt="Transformer layer normalization" height="500" caption="Different arrangements of layer normalization in a transformer encoder layer" src="figures/layer-norm.png?raw=1" id="layer-norm"/>

于是我们有了完整的一个Transformer block的实现

In [21]:
class TransformerEncoderLayer(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.layer_norm_1 = nn.LayerNorm(config.hidden_size)
        self.layer_norm_2 = nn.LayerNorm(config.hidden_size)
        self.attention = MultiHeadAttention(config)
        self.feed_forward = FeedForward(config)

    def forward(self, x):
        # Apply layer normalization and then copy input into query, key, value
        hidden_state = self.layer_norm_1(x)
        # Apply attention with a skip connection
        x = x + self.attention(hidden_state)
        # Apply feed-forward layer with a skip connection
        x = x + self.feed_forward(self.layer_norm_2(x))
        return x

In [22]:
encoder_layer = TransformerEncoderLayer(config)
inputs_embeds.shape, encoder_layer(inputs_embeds).size()

(torch.Size([1, 5, 768]), torch.Size([1, 5, 768]))

## Transformer 的 token 位置嵌入 (positional embeding)

可以发现上面Transformer模块有一个重要的性质，即**各个token的地位是对等的，输出具有token交换不变的属性**。

 - 从操作上看，模块里涉及的token间（同学间）相互交流，和token自己经过的FFN的更新，对每个token都是对等的。
 - 从结果上看，如果把token顺序交换进行输入，即交换两个单词位置，那么输出的特征也不会变，仅位置作了相应调换。

然而在文本/图像处理中，token实际的位置实际包含信息，是需要提供给网络的。因此，需要把位置编码进初始embedding的特征中。常规的操作是：对token自身的编码和位置编码，各自进行embedding，然后相加即可。这就是下面的实现方法。

**注意：** 对于粒子物理的应用中，我们处理的粒子数据，相应于机器学习中“点云识别”的任务，其token本身是地位均等的，因此就没有必要进行位置编码了。

In [23]:
class Embeddings(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.token_embeddings = nn.Embedding(config.vocab_size,
                                             config.hidden_size)
        self.position_embeddings = nn.Embedding(config.max_position_embeddings,
                                                config.hidden_size)
        self.layer_norm = nn.LayerNorm(config.hidden_size, eps=1e-12)
        self.dropout = nn.Dropout()

    def forward(self, input_ids):
        # Create position IDs for input sequence
        seq_length = input_ids.size(1)
        position_ids = torch.arange(seq_length, dtype=torch.long).unsqueeze(0)
        # Create token and position embeddings
        token_embeddings = self.token_embeddings(input_ids)
        position_embeddings = self.position_embeddings(position_ids)
        # Combine token and position embeddings
        embeddings = token_embeddings + position_embeddings
        embeddings = self.layer_norm(embeddings)
        embeddings = self.dropout(embeddings)
        return embeddings

In [ ]:
embedding_layer = Embeddings(config)
embedding_layer(inputs.input_ids).size()

torch.Size([1, 5, 768])

In [ ]:
class TransformerEncoder(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.embeddings = Embeddings(config)
        self.layers = nn.ModuleList([TransformerEncoderLayer(config)
                                     for _ in range(config.num_hidden_layers)])

    def forward(self, x):
        x = self.embeddings(x)
        for layer in self.layers:
            x = layer(x)
        return x

In [ ]:
encoder = TransformerEncoder(config)
encoder(inputs.input_ids).size()

torch.Size([1, 5, 768])

## Transformer 的 class token

经过多层Transformer block，我们依然得到的是各个token自己更新后的特征。如果我们做分类任务，需要把它们合成一个向量。这里有两种做法：

 - 如CNN那样，最后一般进行池化（pooling）操作，例如average pooling或max pooling，把多个向量合为1个。
 - 在Transformer的架构发展中，早期引入的一种更适配self-attention的方案是引入一个专门的class token（记为 `[CLS]`）：它的初始特征随机，但是在Transformer blocks中和各token充分沟通，最后更新的特征直接拿来使用（其它token的特征则都抛弃了）。

这样的class token，最后再经过一层线性变换使维度变为分类头的维度$N_{\rm out}$，即可用来作分类任务的输出。下面是一种实现方式。

In [24]:
class TransformerForSequenceClassification(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.encoder = TransformerEncoder(config)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)

    def forward(self, x):
        x = self.encoder(x)[:, 0, :] # select hidden state of [CLS] token
        x = self.dropout(x)
        x = self.classifier(x)
        return x

In [ ]:
config.num_labels = 3
encoder_classifier = TransformerForSequenceClassification(config)
encoder_classifier(inputs.input_ids).size()

至此，一个完整的Transformer网络就搭建完成啦。在下一节中，下面将利用这个基于PyTorch搭建起的Transformer网络进行粒子物理中分类任务的训练。